In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_f1e86ad7634b44ea92eef84056b442cb.csv"
df=pd.read_csv(url,header=None,na_values=["?"])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [3]:
df.shape

(303, 14)

In [4]:
df.dtypes

0       int64
1       int64
2       int64
3       int64
4       int64
5       int64
6       int64
7       int64
8       int64
9     float64
10      int64
11    float64
12    float64
13      int64
dtype: object

In [5]:
df.isna().sum()/len(df)

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.013201
12    0.006601
13    0.000000
dtype: float64

In [6]:
df=df.fillna(df.median())
df.isna().any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
dtype: bool

In [7]:
df=df.rename(columns={13:"Diagnosis"})
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,Diagnosis
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [8]:
import numpy as np
np.unique(df[["Diagnosis"]])

array([0, 1, 2, 3, 4])

In [9]:
Diagnosis=df[["Diagnosis"]].replace([1,2,3,4],1)
np.unique(Diagnosis)

array([0, 1])

In [10]:
Diagnosis.groupby("Diagnosis")["Diagnosis"].count()

Diagnosis
0    164
1    139
Name: Diagnosis, dtype: int64

In [11]:
y=Diagnosis

In [12]:
X=df.iloc[:,0:13]
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0


In [13]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X))
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.948726,0.686202,-2.251775,0.757525,-0.264900,2.394438,1.016684,0.017197,-0.696631,1.087338,2.274579,-0.711131,0.660004
1,1.392002,0.686202,0.877985,1.611220,0.760415,-0.417635,1.016684,-1.821905,1.435481,0.397182,0.649113,2.504881,-0.890238
2,1.392002,0.686202,0.877985,-0.665300,-0.342283,-0.417635,1.016684,-0.902354,1.435481,1.346147,0.649113,1.432877,1.176752
3,-1.932564,0.686202,-0.165268,-0.096170,0.063974,-0.417635,-0.996749,1.637359,-0.696631,2.122573,2.274579,-0.711131,-0.890238
4,-1.489288,-1.457296,-1.208521,-0.096170,-0.825922,-0.417635,1.016684,0.980537,-0.696631,0.310912,-0.976352,-0.711131,-0.890238


In [14]:

import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
12,112.770163
11,80.800610
8,69.020891
9,66.166707
7,63.419223
2,62.423779
10,39.138194
1,24.978872
0,15.769637
6,8.871394


In [15]:
X_scores=scores[scores["scores"]>1]
len(X_scores)

12

In [16]:
X=X.iloc[:,X_scores.index]

In [17]:
X.head()

,12,11,8,9,7,2,10,1,0,6,3,4
0,0.660004,-0.711131,-0.696631,1.087338,0.017197,-2.251775,2.274579,0.686202,0.948726,1.016684,0.757525,-0.264900
1,-0.890238,2.504881,1.435481,0.397182,-1.821905,0.877985,0.649113,0.686202,1.392002,1.016684,1.611220,0.760415
2,1.176752,1.432877,1.435481,1.346147,-0.902354,0.877985,0.649113,0.686202,1.392002,1.016684,-0.665300,-0.342283
3,-0.890238,-0.711131,-0.696631,2.122573,1.637359,-0.165268,2.274579,0.686202,-1.932564,-0.996749,-0.096170,0.063974
4,-0.890238,-0.711131,-0.696631,0.310912,0.980537,-1.208521,-0.976352,-1.457296,-1.489288,1.016684,-0.096170,-0.825922


In [18]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

In [19]:
BForest=BalancedRandomForestClassifier(n_estimators=210,criterion="gini",max_depth=3,min_samples_split=2,
                                       min_samples_leaf=1,max_features="auto",bootstrap=True,
                                       sampling_strategy="not majority",replacement=True,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest,X,np.asarray(y).reshape(y.shape[0],),cv=8)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8447724039829303 +/- 0.0486415123537297


In [20]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(BForest, X, np.asarray(y).reshape(y.shape[0],), cv=8)
AUC = roc_auc_score(y, y_pred)
print("AUC: ",AUC)

AUC:  0.8380636953851553


In [21]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[151  13]
 [ 34 105]]


In [22]:
y.groupby("Diagnosis")["Diagnosis"].count()

Diagnosis
0    164
1    139
Name: Diagnosis, dtype: int64

In [23]:
print("Number of correctly classified instances: ",151+105)

Number of correctly classified instances:  256


In [24]:
print("Number of misclassified instances: ",34+13)

Number of misclassified instances:  47


In [25]:
y_pred=pd.DataFrame(y_pred).rename(columns={0:"Predicted label"})

In [26]:
y_pred.head()

,Predicted label
0,0
1,1
2,1
3,0
4,0


In [27]:
compare=pd.concat([y,y_pred],axis=1)
compare=pd.DataFrame(compare["Diagnosis"]==compare["Predicted label"]).rename(columns={0:"Comparison"})
comparison=compare[compare["Comparison"]==False]
comparison

,Comparison
16,False
22,False
32,False
33,False
44,False
52,False
56,False
57,False
59,False
66,False


In [28]:
len(comparison)

47

In [29]:
df1=pd.concat([X,y],axis=1)
df1.head()

,12,11,8,9,7,2,10,1,0,6,3,4,Diagnosis
0,0.660004,-0.711131,-0.696631,1.087338,0.017197,-2.251775,2.274579,0.686202,0.948726,1.016684,0.757525,-0.264900,0
1,-0.890238,2.504881,1.435481,0.397182,-1.821905,0.877985,0.649113,0.686202,1.392002,1.016684,1.611220,0.760415,1
2,1.176752,1.432877,1.435481,1.346147,-0.902354,0.877985,0.649113,0.686202,1.392002,1.016684,-0.665300,-0.342283,1
3,-0.890238,-0.711131,-0.696631,2.122573,1.637359,-0.165268,2.274579,0.686202,-1.932564,-0.996749,-0.096170,0.063974,0
4,-0.890238,-0.711131,-0.696631,0.310912,0.980537,-1.208521,-0.976352,-1.457296,-1.489288,1.016684,-0.096170,-0.825922,0


In [30]:
df1.shape

(303, 13)

In [31]:
missed=df1.loc[comparison.index.tolist(),:]
missed.head()

,12,11,8,9,7,2,10,1,0,6,3,4,Diagnosis
16,1.176752,-0.711131,-0.696631,-0.034166,0.805384,-1.208521,2.274579,0.686202,-0.713556,-0.996749,-1.234430,-0.342283,1
22,-0.890238,-0.711131,-0.696631,0.655990,0.455079,-1.208521,0.649113,0.686202,0.394632,1.016684,-0.665300,0.721724,1
32,-0.890238,-0.711131,-0.696631,-0.896862,0.367503,-0.165268,-0.976352,0.686202,1.059545,-0.996749,0.472960,1.708348,1
33,1.176752,-0.711131,-0.696631,-0.465514,0.498867,0.877985,0.649113,0.686202,0.505451,-0.996749,0.188395,-0.245555,0
44,-0.890238,-0.711131,-0.696631,-0.896862,0.849172,0.877985,-0.976352,-1.457296,0.727089,1.016684,-0.096170,1.611621,1


In [32]:
X1=missed.drop(["Diagnosis"],axis=1)
y1=missed[["Diagnosis"]]

In [33]:
BForest1=BalancedRandomForestClassifier(n_estimators=200,criterion="entropy",max_depth=4,min_samples_split=3,
                                       min_samples_leaf=2,max_features="log2",bootstrap=False,
                                       sampling_strategy="not majority",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest1,X1,np.asarray(y1).reshape(y1.shape[0],),cv=8)
print("Accuracy for 2nd round: ",scores.mean(),"+/-",scores.std())

Accuracy for 2nd round:  0.725 +/- 0.07592027982620253


In [34]:
y_pred1 = cross_val_predict(BForest1, X1, np.asarray(y1).reshape(y1.shape[0],), cv=8)
print("Confusion matrix: \n",confusion_matrix(y1,y_pred1))

Confusion matrix: 
 [[ 2 11]
 [ 2 32]]


In [35]:
y1.groupby("Diagnosis")["Diagnosis"].count()

Diagnosis
0    13
1    34
Name: Diagnosis, dtype: int64

In [36]:
y_pred1=pd.DataFrame(y_pred1).rename(columns={0:"Predicted label"})

In [37]:
y1=y1.reset_index(drop=True)

In [38]:
compare=pd.concat([y1,y_pred1],axis=1)
compare=pd.DataFrame(compare["Diagnosis"]==compare["Predicted label"]).rename(columns={0:"Comparison"})
comparison=compare[compare["Comparison"]==False]
comparison

,Comparison
3,False
8,False
10,False
12,False
13,False
14,False
18,False
19,False
21,False
23,False


In [39]:
len(comparison)

13

In [40]:
X1=X1.reset_index(drop=True)

In [41]:
df1=pd.concat([X1,y1],axis=1)
df1.head()

,12,11,8,9,7,2,10,1,0,6,3,4,Diagnosis
0,1.176752,-0.711131,-0.696631,-0.034166,0.805384,-1.208521,2.274579,0.686202,-0.713556,-0.996749,-1.234430,-0.342283,1
1,-0.890238,-0.711131,-0.696631,0.655990,0.455079,-1.208521,0.649113,0.686202,0.394632,1.016684,-0.665300,0.721724,1
2,-0.890238,-0.711131,-0.696631,-0.896862,0.367503,-0.165268,-0.976352,0.686202,1.059545,-0.996749,0.472960,1.708348,1
3,1.176752,-0.711131,-0.696631,-0.465514,0.498867,0.877985,0.649113,0.686202,0.505451,-0.996749,0.188395,-0.245555,0
4,-0.890238,-0.711131,-0.696631,-0.896862,0.849172,0.877985,-0.976352,-1.457296,0.727089,1.016684,-0.096170,1.611621,1


In [42]:
df1.shape

(47, 13)

In [43]:
df2=df1.loc[comparison.index.tolist(),:]
df2

,12,11,8,9,7,2,10,1,0,6,3,4,Diagnosis
3,1.176752,-0.711131,-0.696631,-0.465514,0.498867,0.877985,0.649113,0.686202,0.505451,-0.996749,0.188395,-0.245555,0
8,-0.890238,0.360873,1.435481,0.310912,-1.077507,-2.251775,-0.976352,0.686202,-0.381100,1.016684,-0.380735,-0.651812,0
10,-0.890238,-0.711131,-0.696631,2.208842,-0.114167,-0.165268,0.649113,0.686202,-0.935194,-0.996749,1.042090,-0.303591,1
12,-0.890238,-0.711131,1.435481,1.691225,-0.070379,0.877985,0.649113,0.686202,-1.046013,1.016684,-1.575908,-0.748540,0
13,1.176752,2.504881,-0.696631,0.655990,-0.157955,-0.165268,0.649113,0.686202,0.837908,-0.996749,-0.096170,-0.303591,0
14,-0.890238,0.360873,-0.696631,-0.896862,0.411291,0.877985,-0.976352,-1.457296,0.283813,1.016684,-0.209996,1.089290,0
18,1.176752,0.360873,-0.696631,-0.034166,0.060985,-0.165268,-0.976352,0.686202,1.502821,-0.996749,-0.779126,0.586305,0
19,1.176752,-0.711131,1.435481,-0.896862,-1.690541,0.877985,-0.976352,0.686202,-0.159462,1.016684,0.586786,-0.400319,0
21,1.176752,2.504881,-0.696631,-0.810592,-0.114167,0.877985,-0.976352,0.686202,-0.270281,-0.996749,-1.348256,-0.264900,0
23,1.176752,-0.711131,-0.696631,2.726460,-0.201743,-2.251775,2.274579,0.686202,0.505451,1.016684,2.635654,0.450886,0


In [44]:
X2=df2.drop(["Diagnosis"],axis=1)
y2=df2[["Diagnosis"]]

In [45]:
BForest2=BalancedRandomForestClassifier(n_estimators=200,criterion="entropy",max_depth=4,min_samples_split=3,
                                       min_samples_leaf=2,max_features="log2",bootstrap=False,
                                       sampling_strategy="not majority",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest2,X2,np.asarray(y2).reshape(y2.shape[0],),cv=2)
print("Accuracy for 2nd round: ",scores.mean(),"+/-",scores.std())

Accuracy for 2nd round:  0.8452380952380952 +/- 0.011904761904761862


In [46]:
y_pred2 = cross_val_predict(BForest2, X2, np.asarray(y2).reshape(y2.shape[0],), cv=2)
print("Confusion matrix: \n",confusion_matrix(y2,y_pred2))

Confusion matrix: 
 [[11  0]
 [ 2  0]]


In [47]:
y2.groupby("Diagnosis")["Diagnosis"].count()

Diagnosis
0    11
1     2
Name: Diagnosis, dtype: int64

In [93]:
print("Number of correclty classified instances from round 1: ",256)
print("Number of correclty classified instances from round 2: ",34)
print("Number of correctly classified instances from round 3: ",11)
print("Final accuracy: ",(256+34+11)/303)

Number of correclty classified instances from round 1:  256
Number of correclty classified instances from round 2:  34
Number of correctly classified instances from round 3:  11
Final accuracy:  0.9933993399339934


In [103]:
print("Standard deviation from round 1: +/-",0.0486415123537297)
print("Standard deviation from round 2: +/-",0.07592027982620253)
print("Standard deviation from round 3: +/1",0.011904761904761862)
print("Average standard deviation: +/-",(0.0486415123537297+0.07592027982620253+0.011904761904761862)/3)

Standard deviation from round 1: +/- 0.0486415123537297
Standard deviation from round 2: +/- 0.07592027982620253
Standard deviation from round 3: +/1 0.011904761904761862
Average standard deviation: +/- 0.0454888513615647


After 3 rounds of reinforced targeted learning, the model correctly classified 301 instances out of 303 instances. Very nice result.